In [38]:
import fitbit as fitbit
import pandas as pd 
import numpy as np
import datetime
from datetime import timedelta
import gather_keys_oauth2 as Oauth2
from __future__ import division

import secret

In [29]:
CLIENT_ID = secret.CLIENT_ID
CLIENT_SECRET = secret.CLIENT_SECRET

In [3]:
server = Oauth2.OAuth2Server(CLIENT_ID, CLIENT_SECRET)
server.browser_authorize()

ACCESS_TOKEN = str(server.fitbit.client.session.token['access_token'])
REFRESH_TOKEN = str(server.fitbit.client.session.token['refresh_token'])

##Build Auth Clients
auth2_client = fitbit.Fitbit(CLIENT_ID, CLIENT_SECRET, oauth2=True, access_token=ACCESS_TOKEN, refresh_token=REFRESH_TOKEN, system='en_GB')

#Need this separately because version 1 doesn't give sleep breakdown
auth2_client_new = fitbit.Fitbit(CLIENT_ID, CLIENT_SECRET, oauth2=True, access_token=ACCESS_TOKEN, refresh_token=REFRESH_TOKEN, system='en_GB')
auth2_client_new.API_VERSION = 1.2

[30/Apr/2020:20:50:21] ENGINE Listening for SIGHUP.
[30/Apr/2020:20:50:21] ENGINE Listening for SIGTERM.
[30/Apr/2020:20:50:21] ENGINE Listening for SIGUSR1.
[30/Apr/2020:20:50:21] ENGINE Bus STARTING
CherryPy Checker:
The Application mounted at '' has an empty config.

[30/Apr/2020:20:50:21] ENGINE Started monitor thread 'Autoreloader'.
[30/Apr/2020:20:50:22] ENGINE Serving on http://127.0.0.1:8080
[30/Apr/2020:20:50:22] ENGINE Bus STARTED


127.0.0.1 - - [30/Apr/2020:20:50:25] "GET /?code=07537a761f729e88d5a6eb337e4261e187b4e6ed&state=BtvL8Um4NPE4LuxSAj2oIjncfxXsbM HTTP/1.1" 200 122 "" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36"


[30/Apr/2020:20:50:26] ENGINE Bus STOPPING
[30/Apr/2020:20:50:26] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8080)) shut down
[30/Apr/2020:20:50:26] ENGINE Stopped thread 'Autoreloader'.
[30/Apr/2020:20:50:26] ENGINE Bus STOPPED
[30/Apr/2020:20:50:26] ENGINE Bus EXITING
[30/Apr/2020:20:50:26] ENGINE Bus EXITED
[30/Apr/2020:20:50:26] ENGINE Waiting for child threads to terminate...


In [58]:
DATE_FORMAT = '%Y-%m-%d'
databaseFilename = 'data/database_main.xls'

def connectAndLoadDb():
    print("Connecting database...")
    database = pd.read_excel(databaseFilename)
    print("Database connected!")
    return database;

def connectAndLoadDbFromFile(filename):
    print("Connecting database...")
    database = pd.read_excel(filename)
    print("Database connected!")
    return database;

def getLastEntryDate(database):
    lastDateEntry = database.iloc[-1]['Date']
    lastDateEntry = datetime.datetime.strptime(lastDateEntry, DATE_FORMAT)    
    return lastDateEntry

def addEntriesInDB(dictionary, database):
    #print(dictionary)
    database = database.append(dictionary, ignore_index=True)
    return addEntriesInDB;

def writeDbToExcelFile(database):
    print('Writing database to filename: '+ databaseFilename)
    writer = pd.ExcelWriter(databaseFilename)
    database.to_excel(writer, 'main')
    writer.save()
    print('Database updated with new entries!!')
    
def writeDbToExcelFileSpecified(database, fileName):
    print('Writing database to filename: '+ fileName)
    writer = pd.ExcelWriter(fileName)
    database.to_excel(writer, 'main')
    writer.save()
    print('Database updated with new entries!!')
    
def prettyPrintDate(date):
    return date.strftime(DATE_FORMAT);

In [39]:
def safeGet(obj, key, defaultVal = np.nan):
    return obj.get(key, defaultVal)  

def percent(val):
    return np.ceil(val*100)

def filterArrObj(arrList, keyName, keyValue):
    for arrItem in arrList:
        if arrItem.get(keyName) == keyValue:
            return arrItem
    return {} 

def mergeDicts(dicts):
    super_dict = {}
    for singleDict in dicts:
        for k, v in singleDict.items(): 
            super_dict[k] = v
    return super_dict

    
def getActivities(date):
    activitiyResponse = auth2_client.activities(date=date)
    
    activitySummary = activitiyResponse['summary'];
    activityData = {
        'Calories Burned':safeGet(activitySummary,'caloriesOut'),
        'Calories BMR': safeGet(activitySummary,'caloriesBMR'),
        'Resting Heart Rate':safeGet(activitySummary,'restingHeartRate'),
        'Steps':safeGet(activitySummary,'steps'),
        'Distance (Km)':filterArrObj(activitySummary.get('distances', []), 'activity', 'total').get('distance', np.nan),
        'Minutes Sedentary':activitySummary['sedentaryMinutes'],
        'Minutes Lightly Active':activitySummary['lightlyActiveMinutes'],
        'Minutes Fairly Active':activitySummary['fairlyActiveMinutes'],
        'Minutes Very Active':activitySummary['veryActiveMinutes'],
        'Activity Calories': activitySummary['activityCalories'],
        'Active Score': activitySummary['activeScore'],
        'Cardio minutes': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Cardio').get('minutes', np.nan),
        'Cardio calories': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Cardio').get('caloriesOut', np.nan),
        'Fat Burn minutes': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Fat Burn').get('minutes', np.nan),
        'Fat Burn calories': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Fat Burn').get('caloriesOut', np.nan),
        'Peak minutes': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Peak').get('minutes', np.nan),
        'Peak calories': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Peak').get('caloriesOut', np.nan),
        'Normal Cardio minutes': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Out of Range').get('minutes', np.nan),
        'Normal Cardio calories': filterArrObj(activitySummary.get('heartRateZones', []), 'name', 'Out of Range').get('caloriesOut', np.nan),
    }
    
    return activityData

def getSleep(date):
    sleepResponse = auth2_client_new.sleep(date=date)
    
    sleepData = {}
    for sleepLog in sleepResponse.get('sleep', []):
        if sleepLog.get('isMainSleep'):
            sleepLevelsSummary = sleepLog.get('levels', {}).get('summary', {})
            
            sleepData['Sleep Efficiency'] = safeGet(sleepLog, 'efficiency')
            sleepData['Minutes Asleep'] = safeGet(sleepLog, 'minutesAsleep')
            sleepData['Minutes to fall asleep'] = safeGet(sleepLog, 'minutesToFallAsleep')
            sleepData['Sleep Start time'] = safeGet(sleepLog, 'startTime')
            sleepData['Sleep End time'] = safeGet(sleepLog, 'endTime')
            
            sleepData['Time in bed'] = safeGet(sleepLog, 'timeInBed')
            
            sleepData['Minutes Deep sleep'] = safeGet(sleepLevelsSummary.get('deep', {}), 'minutes')
            sleepData['Deep sleep count'] = safeGet(sleepLevelsSummary.get('deep', {}), 'count')
            sleepData['% Deep sleep'] = percent(sleepData['Minutes Deep sleep']/sleepData['Time in bed'])
            
            sleepData['Minutes Light sleep'] = safeGet(sleepLevelsSummary.get('light', {}), 'minutes')
            sleepData['Light sleep count'] = safeGet(sleepLevelsSummary.get('light', {}), 'count')
            sleepData['% Light sleep'] = percent(sleepData['Minutes Light sleep']/sleepData['Time in bed'])
            
            sleepData['Minutes REM sleep'] = safeGet(sleepLevelsSummary.get('rem', {}), 'minutes')
            sleepData['REM sleep count'] = safeGet(sleepLevelsSummary.get('rem', {}), 'count')
            sleepData['% REM sleep'] = percent(sleepData['Minutes REM sleep']/sleepData['Time in bed'])
            
            sleepData['Minutes Asleep'] = sleepData['Minutes Deep sleep'] + sleepData['Minutes Light sleep'] + sleepData['Minutes REM sleep']
            sleepData['Minutes Awake'] = safeGet(sleepLevelsSummary.get('wake', {}), 'minutes')
            sleepData['% Awake time'] = percent(sleepData['Minutes Awake']/sleepData['Time in bed'])
            sleepData['Minutes Awake count'] = safeGet(sleepLevelsSummary.get('wake', {}), 'count')            
    return sleepData

def getDateData(date):
    weekDayNum = date.isoweekday()
    return {
        'Day of Week': weekDayNum,
        'Is Weekday': weekDayNum<6,
        'Is Weekend': weekDayNum>5,
        'Date': prettyPrintDate(date)
    };

def fetchAllData(date):
    dateStr = prettyPrintDate(date)
    
    print("Fetching fitbit data for: " + dateStr)
    
    nextDate = date + timedelta(days=1)
    sleepData = getSleep(prettyPrintDate(nextDate))
    
    activitiesData = getActivities(dateStr)
    dateData = getDateData(date)
    
    mergedData = mergeDicts([dateData, activitiesData, sleepData])
    return mergedData

In [18]:
def shouldFetchDataForProvidedDate(providedDate, todaysDate, API_COUNTER):
    return (providedDate < todaysDate) and API_COUNTER < 100;

def fetchAndAppendToDb(date, database):    
    mergedData = fetchAllData(date)
    database = database.append(mergedData, ignore_index=True)
    return database;

def fetchData(database, refetchAll = False):
    API_COUNTER = 0
    
    todaysDate = datetime.datetime.today()
    
    print("Date today is :" + prettyPrintDate(todaysDate))
    
    if refetchAll == True:
        lastEntryDate = datetime.datetime.strptime('2020-04-10', DATE_FORMAT)
    else:
        lastEntryDate = getLastEntryDate(database)
    
    print("Last entry in Db is of :" + prettyPrintDate(lastEntryDate))
    print("----------------------------------------------")
    
    dateToFetch = lastEntryDate + timedelta(days=1)

    while shouldFetchDataForProvidedDate(dateToFetch, todaysDate, API_COUNTER):    
        database = fetchAndAppendToDb(dateToFetch, database)
        dateToFetch = dateToFetch + timedelta(days=1)
        API_COUNTER = API_COUNTER+1
    
    print("----------------------------------------------")
    print("Data fill completed! 👍👍")
    return database

Connecting database...
Database connected!


In [54]:
database = connectAndLoadDbFromFile('data/my_database.xls')

database = fetchData(database, refetchAll = True)
# database = fetchData(database)

Connecting database...
Database connected!
Date today is :2020-04-30
Last entry in Db is of :2020-04-10
----------------------------------------------
Fetching fitbit data for: 2020-04-11
Fetching fitbit data for: 2020-04-12
Fetching fitbit data for: 2020-04-13
Fetching fitbit data for: 2020-04-14
Fetching fitbit data for: 2020-04-15
Fetching fitbit data for: 2020-04-16
Fetching fitbit data for: 2020-04-17
Fetching fitbit data for: 2020-04-18
Fetching fitbit data for: 2020-04-19
Fetching fitbit data for: 2020-04-20
Fetching fitbit data for: 2020-04-21
Fetching fitbit data for: 2020-04-22
Fetching fitbit data for: 2020-04-23
Fetching fitbit data for: 2020-04-24
Fetching fitbit data for: 2020-04-25
Fetching fitbit data for: 2020-04-26
Fetching fitbit data for: 2020-04-27
Fetching fitbit data for: 2020-04-28
Fetching fitbit data for: 2020-04-29
Fetching fitbit data for: 2020-04-30
----------------------------------------------
Data fill completed! 👍👍


In [55]:
# re-organize data columns in logical order
columnsTitles = ["Date","Day of Week","Is Weekday","Is Weekend","Calories Burned","Calories BMR","Steps","Distance (Km)","Resting Heart Rate","Minutes Sedentary","Minutes Lightly Active","Minutes Fairly Active","Minutes Very Active","Activity Calories","Cardio minutes","Cardio calories","Fat Burn minutes","Fat Burn calories","Peak minutes","Peak calories","Normal Cardio calories","Normal Cardio minutes","Sleep Efficiency","Minutes Asleep","Minutes to fall asleep","Sleep Start time","Sleep End time","Time in bed","Minutes Deep sleep","Deep sleep count","Minutes Light sleep","Light sleep count","Minutes REM sleep","REM sleep count","Minutes Awake","Minutes Awake count","% Deep sleep","% Light sleep","% REM sleep"]

database = database.reindex(columns=columnsTitles)
database

,Date,Day of Week,Is Weekday,Is Weekend,Calories Burned,Calories BMR,Steps,Distance (Km),Resting Heart Rate,Minutes Sedentary,...,Deep sleep count,Minutes Light sleep,Light sleep count,Minutes REM sleep,REM sleep count,Minutes Awake,Minutes Awake count,% Deep sleep,% Light sleep,% REM sleep
0,2020-04-11,6.0,0.0,1.0,1168.0,1150.0,115.0,0.08,NaN,1364.0,...,2.0,202.0,28.0,56.0,6.0,55.0,30.0,21.0,52.0,15.0
1,2020-04-12,7.0,0.0,1.0,1349.0,1150.0,2024.0,1.36,61.0,924.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-04-13,1.0,1.0,0.0,1624.0,1150.0,6795.0,5.41,60.0,1323.0,...,3.0,170.0,25.0,53.0,7.0,48.0,27.0,20.0,51.0,16.0
3,2020-04-14,2.0,1.0,0.0,1337.0,1150.0,1665.0,1.12,56.0,1008.0,...,3.0,234.0,35.0,104.0,13.0,66.0,39.0,26.0,44.0,20.0
4,2020-04-15,3.0,1.0,0.0,1449.0,1150.0,2709.0,1.82,55.0,769.0,...,4.0,208.0,25.0,77.0,3.0,45.0,24.0,19.0,52.0,19.0
5,2020-04-16,4.0,1.0,0.0,1546.0,1150.0,4829.0,3.63,52.0,890.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2020-04-17,5.0,1.0,0.0,1392.0,1150.0,2622.0,1.76,52.0,1324.0,...,4.0,219.0,30.0,79.0,10.0,64.0,35.0,24.0,47.0,17.0
7,2020-04-18,6.0,0.0,1.0,1407.0,1150.0,3143.0,2.12,50.0,844.0,...,6.0,254.0,39.0,88.0,8.0,72.0,41.0,19.0,50.0,18.0
8,2020-04-19,7.0,0.0,1.0,1391.0,1150.0,2716.0,1.83,48.0,800.0,...,7.0,213.0,23.0,88.0,8.0,46.0,23.0,22.0,49.0,20.0
9,2020-04-20,1.0,1.0,0.0,1370.0,1150.0,2012.0,1.35,48.0,890.0,...,3.0,236.0,30.0,98.0,9.0,82.0,31.0,18.0,47.0,20.0


In [59]:
writeDbToExcelFileSpecified(database, 'data/my_database.xls')

Writing database to filename: data/my_database.xls
Database updated with new entries!!


In [60]:
# database

,Date,Day of Week,Is Weekday,Is Weekend,Calories Burned,Calories BMR,Steps,Distance (Km),Resting Heart Rate,Minutes Sedentary,...,Deep sleep count,Minutes Light sleep,Light sleep count,Minutes REM sleep,REM sleep count,Minutes Awake,Minutes Awake count,% Deep sleep,% Light sleep,% REM sleep
0,2020-04-11,6.0,0.0,1.0,1168.0,1150.0,115.0,0.08,NaN,1364.0,...,2.0,202.0,28.0,56.0,6.0,55.0,30.0,21.0,52.0,15.0
1,2020-04-12,7.0,0.0,1.0,1349.0,1150.0,2024.0,1.36,61.0,924.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-04-13,1.0,1.0,0.0,1624.0,1150.0,6795.0,5.41,60.0,1323.0,...,3.0,170.0,25.0,53.0,7.0,48.0,27.0,20.0,51.0,16.0
3,2020-04-14,2.0,1.0,0.0,1337.0,1150.0,1665.0,1.12,56.0,1008.0,...,3.0,234.0,35.0,104.0,13.0,66.0,39.0,26.0,44.0,20.0
4,2020-04-15,3.0,1.0,0.0,1449.0,1150.0,2709.0,1.82,55.0,769.0,...,4.0,208.0,25.0,77.0,3.0,45.0,24.0,19.0,52.0,19.0
5,2020-04-16,4.0,1.0,0.0,1546.0,1150.0,4829.0,3.63,52.0,890.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2020-04-17,5.0,1.0,0.0,1392.0,1150.0,2622.0,1.76,52.0,1324.0,...,4.0,219.0,30.0,79.0,10.0,64.0,35.0,24.0,47.0,17.0
7,2020-04-18,6.0,0.0,1.0,1407.0,1150.0,3143.0,2.12,50.0,844.0,...,6.0,254.0,39.0,88.0,8.0,72.0,41.0,19.0,50.0,18.0
8,2020-04-19,7.0,0.0,1.0,1391.0,1150.0,2716.0,1.83,48.0,800.0,...,7.0,213.0,23.0,88.0,8.0,46.0,23.0,22.0,49.0,20.0
9,2020-04-20,1.0,1.0,0.0,1370.0,1150.0,2012.0,1.35,48.0,890.0,...,3.0,236.0,30.0,98.0,9.0,82.0,31.0,18.0,47.0,20.0
